In [1]:
import json
import pandas as pd
# import numpy as np
import datetime as dt

In [2]:
ldr_date = dt.date(2018, 9, 16)

In [10]:
text_file = 'ChatExport/result.json'
with open(text_file, encoding="utf8") as f:
    d = json.load(f)
cols = ['type','date','from','from_id','text','sticker_emoji','file','media_type','photo','action']
text_df = pd.json_normalize(d['messages'])[cols]

# Clean data and create new columns

In [11]:
text_df = text_df[text_df['action']!='phone_call']
text_df['from_id'] = text_df['from_id'].astype('str')
text_df['datetime'] = pd.to_datetime(text_df['date'])

text_df['date'] = text_df['datetime'].dt.date

text_df['hour'] = text_df['datetime'].dt.hour

text_df['from'] = np.where(text_df['from']=='Trường Hoàng',
                           u'\U0001F425'+u'\U0001F49B'+u'\U0001F33B', text_df['from'])

text_df['text2'] = text_df['text'].replace('[\'!#$%&()*+,-./:;<=>?@^_`{|}~]', '', regex=True)
text_df['text2'] = text_df['text2'].replace('\\s+', ' ', regex=True)

text_df['word_count'] = text_df['text2'].str.split().str.len()

dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
text_df['weekday_id'] = text_df['datetime'].dt.dayofweek
text_df['weekday'] = text_df['weekday_id'].map(dayOfWeek)

text_df['isLDR'] = np.where(text_df['date'] <= ldr_date,
                            0, 1)

day_intervals = [text_df['hour'].between(6, 9),
                 text_df['hour'].between(10, 13),
                 text_df['hour'].between(14, 17),
                 text_df['hour'].between(18, 21),
                 text_df['hour'].between(22, 24),
                 text_df['hour'].between(0, 5)]
day_parts = ['morning_1','morning_2','afternoon','evening_1','evening_2','night']

text_df['daypart'] = np.select(day_intervals, day_parts, 0)

text_df['datetime_eu'] = text_df['datetime'].dt.tz_localize(tz='Asia/Ho_Chi_Minh').dt.tz_convert('Europe/Berlin').dt.tz_localize(None)

text_df['hour_eu'] = text_df['datetime_eu'].dt.hour
day_intervals_eu = [text_df['hour_eu'].between(6, 9),
                 text_df['hour_eu'].between(10, 13),
                 text_df['hour_eu'].between(14, 17),
                 text_df['hour_eu'].between(18, 21),
                 text_df['hour_eu'].between(22, 24),
                 text_df['hour_eu'].between(0, 5)]
text_df['daypart_eu'] = np.select(day_intervals_eu, day_parts,0)

text_df['isNight'] = np.where((text_df['daypart']=='night') & (text_df['daypart_eu']=='night'),
                              1, 0)

text_df['datetime_next'] = np.where(text_df['datetime']==max(text_df['datetime']),
                                    text_df['datetime'],
                                    text_df['datetime'].shift(-1))

text_df['buffer'] = (text_df['datetime_next'] - text_df['datetime']).dt.seconds/(60*60*24)

text_df['sticker_img'] = np.where(text_df['media_type']=='sticker', 
                                   'ChatExport/' + text_df['file'],
                                   np.nan)

In [12]:
text_df.to_csv('text_df.csv')

# Visualizations

In [13]:
from IPython.display import Image, HTML

df = text_df[text_df['media_type']=='sticker'].groupby(['sticker_img','from']).agg({'from_id':'count'}).reset_index()
df = df.pivot(index='sticker_img',columns='from',values='from_id').reset_index()
df['total'] = df['🐑💛🌙'] + df['🐥💛🌻']
df = df.sort_values(['total'], ascending=False).head(10)

def path_to_image_html(path):
    return '<img src="'+ path + '" width="30" height="30"/>'

pd.set_option('display.max_colwidth', None)

HTML(df.to_html(escape=False ,formatters=dict(sticker_img=path_to_image_html)))

from,sticker_img,🐑💛🌙,🐥💛🌻,total
280,,313.0,529.0,842.0
182,,236.0,445.0,681.0
193,,219.0,273.0,492.0
304,,21.0,447.0,468.0
293,,82.0,334.0,416.0
605,,185.0,166.0,351.0
601,,195.0,152.0,347.0
292,,20.0,319.0,339.0
286,,96.0,215.0,311.0
285,,72.0,221.0,293.0
